Importing
--

In [1]:
import os
from PIL import Image
import matplotlib.colors as plt
from matplotlib.pyplot import imshow
import webcolors as wc
from imageio import imread
from sklearn.cluster import KMeans
from scipy.spatial import KDTree
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

## Resizing Images and Converting

In [2]:
# images = os.listdir(r"D:\Pics Training\Pics Png")
# i = 1
# root_path = "D:\Pics Training"

# for image in images:
#     path = os.path.join(root_path, image)
#     img = Image.open(path)
#     img = img.resize((200, 200))
#     img.save("D:\\Pics Training\\Pics Png\\" + str(i) + ".png")
#     i = i+1
#     img.close()
#     if i==40:
#         break

Converting RGB to Name
--

In [3]:
def convert_rgb_to_names(rgb_tuple):
    css3_db = wc.CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(wc.hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

Finding Image Colors
--

In [ ]:
images = os.listdir(r"D:\Pics Training\Pics Png")
root_path = "D:\Pics Training\Pics png"
colors = []

k=5
kmeans = KMeans(n_clusters=k)

for image in images:
    path = os.path.join(root_path, image)
    img = imread(path)
    img = img.reshape(-1, 3)
    kmeans.fit(img)
    clusters = kmeans.cluster_centers_
    color_names=[]
    for row in clusters:
        color_names.append(convert_rgb_to_names(row))
    colors.append(color_names)

Creatin the Data Set by Mapping
-

In [5]:
all_color_names = list(wc.CSS3_HEX_TO_NAMES.values())

def color_matching(image_colors, names_list=all_color_names):
    all_image_colors = []
    for ic in image_colors:
        color_mapping = [-1 for i in range(len(names_list))]
        for index, color in enumerate(names_list):
            if ic[0] == color:
                color_mapping[index] = 1 
            elif ic[1] == color:
                color_mapping[index] = 1 
            elif ic[2] == color:
                color_mapping[index] = 1 
            elif ic[3] == color:
                color_mapping[index] = 1 
            elif ic[4] == color:
                color_mapping[index] = 1 
            else:
                color_mapping[index] = 0
        all_image_colors.append(color_mapping)
    return all_image_colors

mapped = color_matching(colors)

Preparing Data
--

In [6]:
data = pd.DataFrame(mapped, columns=all_color_names)
features = data.copy()
pd.set_option('display.max_columns', None)

In [7]:
data['pic'] = ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 
               'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S']
target = data['pic'].map({'M':1, 'S':0})

Test and Train Data
-

In [8]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.2, random_state=42)

Random Forest
-

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=10, criterion='entropy', max_depth=3, max_samples=10, random_state=34567)
rf_clf.fit(xTrain, yTrain)
yPred = rf_clf.predict(xTest)
accuracy_score(yPred, yTest)

0.875

Linear SVM
-

In [10]:
from sklearn.svm import LinearSVC
lin_svc = LinearSVC(random_state=0)
lin_svc.fit(xTrain, yTrain)
yPred = lin_svc.predict(xTest)
accuracy_score(yPred, yTest)

0.75

Non-Linear SVM
-

In [11]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', random_state=0)
svc.fit(xTrain, yTrain)
yPred = svc.predict(xTest)
accuracy_score(yPred, yTest)

0.875

Logistic Regression
-

In [12]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(xTrain, yTrain)
yPred = log_reg.predict(xTest)
accuracy_score(yTest, yPred)

0.625

Testing on New Instance
-

In [23]:
new_path = r"C:\Users\gowda\Downloads\Untitled.png"

In [24]:
instance = new_color_match(new_path)
prediction = lin_svc.predict([instance])[0]
if prediction:
    print("The new image is a 'Mountain'")
else:
    print("The new image is a 'Sea'")

The new image is a 'Sea'


In [15]:
def new_color_match(path, names_list=all_color_names):
    
    img_col = clasify(path)
    mapping = [-1 for i in range(len(names_list))]

    for ic in img_col:
        for index, color_name in enumerate(names_list):
            if color_name == ic:
                mapping[index] = 1
            elif (color_name != ic and mapping[index]==1):
                mapping[index] = 1
            else:
                mapping[index] =0
    return mapping

In [16]:
def clasify(path):
    colors = []
    kmeans = KMeans(n_clusters=5)
    image = imread(path)
    img = image.reshape(-1, 3)
    kmeans.fit(img)
    clusters = kmeans.cluster_centers_
    for color in clusters:
        colors.append(convert_rgb_to_names(color))
    return colors

In [4]:
colors = [['gray', 'darkslategray', 'lavender', 'lightskyblue', 'dimgray'],
 ['skyblue', 'dimgray', 'lavender', 'darkslategray', 'darkgray'],
 ['darkslategray', 'white', 'gray', 'dimgray', 'dimgray'],
 ['darkslategray', 'black', 'sienna', 'lightslategray', 'darkolivegreen'],
 ['sienna', 'darkslategray', 'dimgray', 'silver', 'steelblue'],
 ['dimgray', 'ghostwhite', 'darkslategray', 'silver', 'lightslategray'],
 ['darkolivegreen', 'darkkhaki', 'darkolivegreen', 'lightslategray', 'silver'],
 ['dimgray', 'lavender', 'darkslategray', 'darkgray', 'black'],
 ['darkolivegreen', 'steelblue', 'darksalmon', 'darkslategray', 'peru'],
 ['cornflowerblue', 'darkslategray', 'tan', 'dimgray', 'lightsteelblue'],
 ['dimgray', 'lavender', 'sienna', 'darkslategray', 'lightblue'],
 ['dimgray', 'white', 'darkgray', 'darkslategray', 'gray'],
 ['darkolivegreen', 'slategray', 'dodgerblue', 'darkslategray', 'darkgray'],
 ['darkslategray', 'lightsteelblue', 'dimgray', 'gray', 'darkslategray'],
 ['darkolivegreen', 'paleturquoise', 'black', 'darkolivegreen', 'gray'],
 ['lightsteelblue', 'darkolivegreen', 'dimgray', 'black', 'powderblue'],
 ['gray', 'darkslategray', 'lightsteelblue', 'darkslategray', 'dimgray'],
 ['lightslategray', 'darkslategray', 'gray', 'darkslategray', 'lightgray'],
 ['darkolivegreen', 'gainsboro', 'darkslategray', 'dimgray', 'darkgray'],
 ['gainsboro', 'steelblue', 'darkolivegreen', 'dimgray', 'darkgray'],
 ['dodgerblue', 'burlywood', 'powderblue', 'skyblue', 'lightslategray'],
 ['steelblue', 'lightslategray', 'darkslategray', 'rosybrown', 'slategray'],
 ['gray', 'lightcyan', 'darkslategray', 'darkolivegreen', 'lightblue'],
 ['peru', 'silver', 'mediumaquamarine', 'darkolivegreen', 'cadetblue'],
 ['lightsteelblue', 'darkslategray', 'steelblue', 'darkcyan', 'cadetblue'],
 ['lightgray', 'deepskyblue', 'tan', 'darkcyan', 'skyblue'],
 ['dodgerblue', 'gainsboro', 'skyblue', 'dodgerblue', 'cornflowerblue'],
 ['cornflowerblue', 'dodgerblue', 'tan', 'lightsteelblue', 'teal'],
 ['royalblue', 'darkslateblue', 'lightblue', 'midnightblue', 'cornflowerblue'],
 ['deepskyblue', 'skyblue', 'steelblue', 'lavender', 'mediumturquoise'],
 ['cornflowerblue', 'darkslateblue', 'lightsteelblue', 'cadetblue', 'darkslategray'],
 ['cornflowerblue', 'tan', 'teal', 'lightsteelblue', 'dodgerblue'],
 ['lightgray', 'darkslategray', 'darkgray', 'midnightblue', 'dimgray'],
 ['lavender', 'steelblue', 'lightblue', 'cornflowerblue', 'teal'],
 ['lightslategray', 'burlywood', 'dodgerblue', 'powderblue', 'skyblue'],
 ['cadetblue', 'lavender', 'darkgray', 'dimgray', 'skyblue'],
 ['dimgray', 'burlywood', 'rosybrown', 'skyblue', 'lightgray'],
 ['gainsboro', 'silver', 'tan', 'lightslategray', 'wheat'],
 ['darkolivegreen', 'lavender', 'mediumturquoise', 'burlywood', 'skyblue'],
 ['lightgray', 'dimgray', 'linen', 'darkgray', 'darkkhaki']]